In [1]:
project_name = 'experiment_5'

In [2]:
import sys
import os
import logging
import mlflow

notebook_dir = os.path.dirname(os.path.abspath('__file__'))
src_dir = os.path.join(notebook_dir, '../../')
sys.path.append(src_dir)

logger = logging.getLogger(__name__)
logging.basicConfig(filename=f'../../logs/{project_name}.log',
                    level=logging.INFO,
                    format="[%(asctime)s] %(levelname)s [%(name)s.%(funcName)s:%(lineno)d] %(message)s",
                    datefmt="%d/%b/%Y %H:%M:%S"
                   )

mlflow.set_experiment(project_name)
mlflow.autolog()

2025/02/27 23:19:24 INFO mlflow.tracking.fluent: Experiment with name 'experiment_5' does not exist. Creating a new experiment.


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from src.metrics.metrics import compute_metrics, default_regression_metrics
from src.preprocessing.preprocessing import Preprocessor
from src.io.input import load_artifacts
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.preprocessing import StandardScaler
from src.pipelines.training_pipelines import StandardTrainerPipeline
from src.train.training import train_multiple_models
from src.models.default import get_default_regression_models
from src.train.cross_validation import cross_validate
from src.metrics.metrics import summarize_cv_metrics
from src.pipelines.hypertune_pipelines import HyperTunnerPipeline
from src.hypertune.hypertuning import objective_score
from hyperopt import hp
#from src.monitor.default_drift import get_default_drift_report

import warnings
warnings.filterwarnings('ignore')

2025/02/27 23:19:25 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/02/27 23:19:25 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
2025/02/27 23:19:25 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.


In [4]:
df = pd.read_csv('../../data/training_data.csv')
print(df.shape)

(800, 21)


In [5]:
df.head(2)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,target
0,432.475954,289.373016,481.315600,358.755566,802.659004,176.761177,72.648102,720.969179,36.327684,83.768878,...,4.385848,516.789458,19.624422,13.162440,42.351948,35.920392,20.755984,13.814300,384.497136,14.364922
1,517.596250,330.448341,585.920055,22.684031,169.813240,335.601640,284.451476,748.101047,73.701438,358.147215,...,5.563334,2.960064,20.721878,17.740184,1.726915,167.576065,75.492679,2.480979,303.710869,19.984801


### 1. Exploratory Data Analysis

In [6]:
#report = ProfileReport(df)
#report.to_file('report.html')

In [7]:
df_train, df_val = train_test_split(df, test_size=0.2)
print(df_train.shape, df_val.shape)

(640, 21) (160, 21)


### 2. Preprocessing

In [8]:
# Preprocessing params
target_column = 'target'
numerical_features = list(df_train.drop(columns=[target_column]).columns)
categorical_features = []
preprocessing_graph = ['individual_num', 'combinated_num']
artifacts = {}
# Fit preprocessors in training mode
train_pipeline = Preprocessor(df_train,
                              'train',
                              numerical_features=numerical_features,
                              categorical_features=categorical_features,
                              target_column=target_column,
                              graph_preprocess=preprocessing_graph,
                              artifacts=artifacts
                             )
df_train_preproc = train_pipeline.preprocess()

# Load generated artifacts
artifacts = load_artifacts('../../artifacts/')
# Apply same preprocess logic to validation set
validation_pipeline = Preprocessor(df_val,
                                   'val',
                                   numerical_features=numerical_features,
                                   categorical_features=categorical_features,
                                   target_column=target_column,
                                   graph_preprocess=preprocessing_graph,
                                   artifacts=artifacts
                                  )
df_val_preproc = validation_pipeline.preprocess()

### 3. Metrics choice

In [9]:
metrics_config = default_regression_metrics()

### 4. Model training

In [10]:
list_models = [XGBRegressor(), LGBMRegressor(verbose=-1),
               RandomForestRegressor(), LinearRegression(),
               Ridge(), Lasso()#, create_neural_network(df_train_preproc.shape[1]-1),
               ]
list_model_names = ['xgboost', 'lightgbm', 'random_forest',
                    'linear_regression', 'ridge', 'lasso'#, 'dnn'
                   ]

metrics_df, fitted_models = train_multiple_models(df_train_preproc,
                                                  df_val_preproc,
                                                  list_models,
                                                  list_model_names,
                                                  target_column,
                                                  metrics_config
                                                 )

2025/02/27 23:19:50 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '3fe03fc1ec0a400cb39618dd20b4f213', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
2025/02/27 23:20:54 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/wh/3hncd7zs7nxdl_qb9d23vr9w0000gn/T/tmptas7jy38/model, flavor: xgboost). Fall back to return ['xgboost==2.1.3']. Set logging level to DEBUG to see the full traceback. 
2025/02/27 23:21:05 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '25c5d5ff2e7f4db88dcf134e1e428c82', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current lightgbm workflow
2025/02/27 23:24:04 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '0d294d2e1e4b4967a857bc110f0b255c', which will track hyperparameters,

In [11]:
metrics_df

,model,dataset,mae,mse,rmse,mape,r2
0,xgboost,train,2.072425e-04,6.835532e-08,2.614485e-04,2.064265e-05,1.000000
0,xgboost,validation,1.806035e+00,5.284825e+00,2.298875e+00,1.428545e-01,0.788489
0,lightgbm,train,4.576032e-02,1.888961e-02,1.374395e-01,1.310353e-02,0.999273
0,lightgbm,validation,1.581541e+00,4.046624e+00,2.011622e+00,1.285436e-01,0.838044
0,random_forest,train,7.759161e-01,9.600577e-01,9.798254e-01,8.085060e-02,0.963058
0,random_forest,validation,1.884376e+00,5.765850e+00,2.401218e+00,1.539849e-01,0.769237
0,linear_regression,train,3.622442e-09,2.337020e-17,4.834273e-09,3.023829e-10,1.000000
0,linear_regression,validation,6.242786e+00,2.588877e+02,1.608999e+01,5.230357e-01,-9.361307
0,ridge,train,9.947995e-01,1.650402e+00,1.284680e+00,9.054498e-02,0.936494
0,ridge,validation,3.892876e+00,9.914693e+01,9.957255e+00,3.073648e-01,-2.968099


### 5. Complete training pipeline

In [12]:
target_column = 'target'
numerical_features = list(df.drop(columns=[target_column]).columns)
categorical_features = []
preprocessing_graph = ['individual_num']#, 'combinated_num']
artifacts = {}
test_size = 0.2

list_models, list_model_names = get_default_regression_models()
metrics_config = default_regression_metrics()

training_pipeline = StandardTrainerPipeline(df.copy(),
                                            numerical_features,
                                            categorical_features,
                                            target_column,
                                            preprocessing_graph,
                                            artifacts.copy(),
                                            list_models.copy(),
                                            list_model_names,
                                            metrics_config,
                                            test_size
                                           )
training_pipeline.run()

2025/02/27 23:29:51 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'c4312892e9f84886aab44a3f749612c2', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
2025/02/27 23:29:54 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/wh/3hncd7zs7nxdl_qb9d23vr9w0000gn/T/tmpgdxu1h89/model, flavor: xgboost). Fall back to return ['xgboost==2.1.3']. Set logging level to DEBUG to see the full traceback. 
2025/02/27 23:29:54 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '86be6eecfc784117b7c770d0093477d4', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current lightgbm workflow
2025/02/27 23:29:58 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'baa192a666344becba4dbc8445583dd8', which will track hyperparameters,

In [13]:
training_pipeline.metrics_df

,model,dataset,mae,mse,rmse,mape,r2
0,xgboost,train,0.000545,5.986409e-07,0.000774,0.000042,1.000000
0,xgboost,validation,1.912291,5.696732e+00,2.386783,0.167361,0.761718
0,lightgbm,train,0.119704,3.834291e-02,0.195813,0.015717,0.998545
0,lightgbm,validation,1.567390,3.704633e+00,1.924742,0.134581,0.845043
0,random_forest,train,0.843664,1.157487e+00,1.075866,0.088283,0.956084
0,random_forest,validation,2.022006,6.390756e+00,2.527994,0.176872,0.732689
0,linear_regression,train,1.266085,2.737394e+00,1.654507,0.110059,0.896141
0,linear_regression,validation,1.970527,7.788063e+00,2.790710,0.163299,0.674242
0,ridge,train,1.366923,3.074664e+00,1.753472,0.124462,0.883344
0,ridge,validation,1.687121,4.900097e+00,2.213616,0.144087,0.795040


### 6. Cross-validation

In [14]:
target_column = 'target'
numerical_features = list(df.drop(columns=[target_column]).columns)
categorical_features = []
preprocessing_graph = ['individual_num']#, 'combinated_num']
artifacts = {}
test_size = 0.2
num_folds = 5

list_models, list_model_names = get_default_regression_models()
metrics_config = default_regression_metrics()

metrics_df, all_models = cross_validate(df.copy(),
                                        list_models,
                                        list_model_names,
                                        target_column,
                                        metrics_config,
                                        num_folds,
                                        artifacts,
                                        numerical_features,
                                        categorical_features,
                                        preprocessing_graph,
                                        test_size
                                       )

2025/02/27 23:30:09 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '96127c3b676b4529bc385a415b9243ef', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
2025/02/27 23:30:12 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/wh/3hncd7zs7nxdl_qb9d23vr9w0000gn/T/tmpkcs1i3cf/model, flavor: xgboost). Fall back to return ['xgboost==2.1.3']. Set logging level to DEBUG to see the full traceback. 
2025/02/27 23:30:12 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'a1208327b3e7462686a1bf8bebe74886', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current lightgbm workflow
2025/02/27 23:30:15 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '42e8d05d538047dd9cf089986e929941', which will track hyperparameters,

In [15]:
summarize_cv_metrics(metrics_df, 'mape')

,dataset,model,mae_cv,mse_cv,rmse_cv,mape_cv,r2_cv
,,,,,,,
5,train,xgboost,0.0009 +/- 0.0005,0.0 +/- 0.0,0.0012 +/- 0.0007,0.0001 +/- 0.0,1.0 +/- 0.0
1,train,lightgbm,0.1173 +/- 0.0031,0.0396 +/- 0.0035,0.1989 +/- 0.0085,0.0149 +/- 0.0036,0.9985 +/- 0.0002
3,train,random_forest,0.8203 +/- 0.0067,1.1015 +/- 0.034,1.0494 +/- 0.0162,0.0819 +/- 0.0069,0.9579 +/- 0.0007
2,train,linear_regression,1.2375 +/- 0.0222,2.5967 +/- 0.0492,1.6114 +/- 0.0153,0.1054 +/- 0.005,0.9008 +/- 0.0017
4,train,ridge,1.3459 +/- 0.0237,2.9658 +/- 0.0565,1.7221 +/- 0.0164,0.1207 +/- 0.0068,0.8866 +/- 0.0027
0,train,lasso,1.5394 +/- 0.0289,3.9031 +/- 0.1081,1.9755 +/- 0.0272,0.1376 +/- 0.0057,0.8508 +/- 0.0048
6,validation,lasso,1.7045 +/- 0.0721,6.1551 +/- 2.5046,2.446 +/- 0.4639,0.1536 +/- 0.019,0.7481 +/- 0.1044
10,validation,ridge,1.7975 +/- 0.0401,6.9687 +/- 3.066,2.597 +/- 0.5294,0.1734 +/- 0.036,0.7145 +/- 0.1281
7,validation,lightgbm,1.747 +/- 0.0476,5.0478 +/- 0.3888,2.2454 +/- 0.0874,0.1813 +/- 0.0463,0.7924 +/- 0.0291


### 6. Hyperparameter tunning

In [16]:
target_column = 'target'
numerical_features = list(df.drop(columns=[target_column]).columns)
categorical_features = []
preprocessing_graph = ['individual_num']#, 'combinated_num']
artifacts = {}
test_size = 0.2
optimized_metric = 'mse'
model2tune = LGBMRegressor
model_name = 'lgbm'
max_iterations = 20

list_models, list_model_names = get_default_regression_models()
metrics_config = default_regression_metrics()

search_space = {}
search_space['max_depth'] = hp.uniformint('max_depth', 1, 500, q=1)
search_space['num_leaves'] = hp.uniformint('num_leaves', 30, 1000, q=1)
search_space['boosting_type'] = hp.choice('boosting_type', ['gbdt', 'dart'])
search_space['colsample_bytree'] = hp.uniform('colsample_bytree', 0.6, 1.0)
search_space['learning_rate'] = 0.1
search_space['reg_alpha'] = hp.uniform('reg_alpha', 0.0, 1000.0)
search_space['reg_lambda'] = hp.uniform('reg_lambda', 0.0, 1000.0)
search_space['verbose'] = -1



hypertune_pipeline = HyperTunnerPipeline(df,
                                        numerical_features,
                                        categorical_features,
                                        target_column,
                                        preprocessing_graph,
                                        artifacts,
                                        search_space,
                                        optimized_metric,
                                        model2tune,
                                        model_name,
                                        metrics_config,
                                        objective_score,
                                        test_size,
                                        max_iterations
                                        )
hypertune_pipeline.run()

  0%|                                      | 0/20 [00:00<?, ?trial/s, best loss=?]

2025/02/27 23:40:21 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '5ac83188b9b944dc97fa03e1c96dd45d', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current lightgbm workflow



{'mae': 2.7269087338056, 'mse': 11.354900249288036, 'rmse': 3.369703288019293, 'mape': 0.24368422913658994, 'r2': 0.515718159843556}
  5%|▌           | 1/20 [00:03<01:00,  3.16s/trial, best loss: 11.354900249288036]

2025/02/27 23:40:24 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'bc84aaf952c74779acf140f30ce0985f', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current lightgbm workflow



{'mae': 21.95028543820006, 'mse': 505.96963843627225, 'rmse': 22.4937688802093, 'mape': 1.5458673824791191, 'r2': -20.579397633244056}
 10%|█▏          | 2/20 [00:06<00:56,  3.16s/trial, best loss: 11.354900249288036]

...

{'mae': 21.93641265797064, 'mse': 505.43733939348095, 'rmse': 22.481933622210544, 'mape': 1.5441839087727163, 'r2': -20.55669529731005}
 90%|██████████▊ | 18/20 [00:58<00:06,  3.30s/trial, best loss: 7.884987564000727]

2025/02/27 23:41:19 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '1ddc17be0f394621bcc09b31ef4711c6', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current lightgbm workflow



{'mae': 21.95598040272435, 'mse': 506.1229422417964, 'rmse': 22.49717631708025, 'mape': 1.5465842291893868, 'r2': -20.585935977695552}
 95%|███████████▍| 19/20 [01:01<00:03,  3.21s/trial, best loss: 7.884987564000727]

2025/02/27 23:41:22 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '6e3dec57f50f4e3d9de8008c93adc77c', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current lightgbm workflow



{'mae': 4.056373147003034, 'mse': 23.56567864707521, 'rmse': 4.854449366001793, 'mape': 0.35285800722347427, 'r2': -0.005066532438853688}
100%|████████████| 20/20 [01:04<00:00,  3.22s/trial, best loss: 7.884987564000727]


In [17]:
hypertune_pipeline.best_hyperparams

{'boosting_type': 0,
 'colsample_bytree': 0.6852279340179045,
 'max_depth': 149.0,
 'num_leaves': 525.0,
 'reg_alpha': 63.61983523606229,
 'reg_lambda': 343.9529944500177}

In [18]:
hypertune_pipeline.trials.best_trial

{'state': 2,
 'tid': 9,
 'spec': None,
 'result': {'loss': 7.884987564000727, 'status': 'ok'},
 'misc': {'tid': 9,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'boosting_type': [9],
   'colsample_bytree': [9],
   'max_depth': [9],
   'num_leaves': [9],
   'reg_alpha': [9],
   'reg_lambda': [9]},
  'vals': {'boosting_type': [0],
   'colsample_bytree': [0.6852279340179045],
   'max_depth': [149.0],
   'num_leaves': [525.0],
   'reg_alpha': [63.61983523606229],
   'reg_lambda': [343.9529944500177]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2025, 2, 28, 4, 40, 50, 222000),
 'refresh_time': datetime.datetime(2025, 2, 28, 4, 40, 53, 417000)}

### 7. Drift evaluation

In [ ]:
df = pd.read_csv('../../data/training_data.csv')
df_test = pd.read_csv('../../data/blind_test_data.csv')

drift_report = get_default_drift_report(df, df_test)
drift_report